In [1]:
import csv
import pandas as pd
import requests
import gmaps

from config import g_key 

In [2]:
caproviderdata = pd.read_csv("Enrolled_Medi-Cal_Fee_For_Service_Provider_File.csv", low_memory=False)
caprovider_df = pd.DataFrame(caproviderdata)

# "C:\Users\Kdw86\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.
# Specify dtype option on import or set low_memory=False.
# has_raised = await self.run_ast_nodes(code_ast.body, cell_name,"

#after warning check and toss invalid numbers (with alphas)
#for i, row in caprovider_df.iterrows():
#    try:
#        row['Provider_Number'] = int(row['Provider_Number'])
#    except:
#        caprovider_df.drop(i, inplace=True)

#caprovider_df.dropna(inplace=True)
#caprovider_df.reset_index(drop=True, inplace=True) 



caprovider_df.head()

,X,Y,Provider_Number,NPI_Provider_Number,Owner_Number,Service_Location_Number,Provider_Legal_Name,Enrollment_Status_Effective_Dat,Provider_Type_Code,Provider_Type_Code_Desc,...,Provider_Address_Line_2,Provider_Address_City,Provider_Address_State,Provider_Address_Zip,Provider_Address_Latitude,Provider_Address_Longitude,MSSA_ID,Tract_ID,File_,ObjectId
0,-117.370428,34.133257,1003120452,1.003120e+09,1.0,1.0,"PETERSON, KIMBERLY OD INC",2010/08/02 07:00:00+00,20,OPTOMETRISTS (FFS),...,SUITE 100,RIALTO,CA,923768071,34.133257,-117.370428,151h,38.01,Fee-for-Service,1
1,-117.152090,32.884362,1003121385,1.003121e+09,1.0,1.0,COUNTRYWIDE MEDICAL SUPPLIES,2011/07/05 07:00:00+00,2,ASSISTIVE DEVICE AND SICK ROOM SUPPLY DEALERS ...,...,SUITE 5300A,SAN DIEGO,CA,921264240,32.884362,-117.152090,161o,94.00,Fee-for-Service,2
2,-118.744681,34.288645,1003207002,1.003207e+09,1.0,2.0,CHANGE WITHIN REACH INC,2018/07/23 07:00:00+00,71,LICENSED CLINICAL SOCIAL WORKER GROUP (FFS),...,STE 24,SIMI VALLEY,CA,930651224,34.288645,-118.744681,240a,79.03,Fee-for-Service,3
3,-124.142048,40.783220,1003121930,1.003122e+09,1.0,1.0,"ALHARIRI, AHMAD",2018/11/27 08:00:00+00,26,PHYSICIANS (FFS),...,NaN,EUREKA,CA,955014736,40.783220,-124.142048,39,6.00,Fee-for-Service,4
4,-117.306377,33.181302,1003122086,1.003122e+09,1.0,1.0,"POLLACK, AARON",2010/10/19 07:00:00+00,13,HEARING AID DISPENSERS (FFS),...,NaN,OCEANSIDE,CA,920564522,33.181302,-117.306377,156b,185.17,Fee-for-Service,5


In [3]:
caprovider_df = caprovider_df.drop(columns=['X', 'Y', "NPI_Provider_Number", "Owner_Number", 'Provider_Address_Zip',
                                            "Service_Location_Number", "Provider_Legal_Name", 
                                            "Provider_Type_Code", "Provider_License_Number",
                                            "Provider_Specialty_Code", "Out_Of_State_Ind", "Out_Of_State_Desc", 
                                            "Provider_Address_County_Code", "Provider_Address_Attention_Line", 
                                            "Provider_Address_Line_1", "Provider_Address_Line_2", "Provider_Address_State", 
                                            "MSSA_ID","Tract_ID", "File_", "ObjectId","Enrollment_Status_Effective_Dat",
                                            "Provider_Address_City"])

caprovider_df.head()   

,Provider_Number,Provider_Type_Code_Desc,Provider_Specialty_Code_Desc,Provider_Address_County_Code_De,Provider_Address_Latitude,Provider_Address_Longitude
0,1003120452,OPTOMETRISTS (FFS),NOT APPLICABLE,San Bernardino,34.133257,-117.370428
1,1003121385,ASSISTIVE DEVICE AND SICK ROOM SUPPLY DEALERS ...,NOT APPLICABLE,San Diego,32.884362,-117.152090
2,1003207002,LICENSED CLINICAL SOCIAL WORKER GROUP (FFS),NOT APPLICABLE,Ventura,34.288645,-118.744681
3,1003121930,PHYSICIANS (FFS),Internal Medicine,Humboldt,40.783220,-124.142048
4,1003122086,HEARING AID DISPENSERS (FFS),NOT APPLICABLE,San Diego,33.181302,-117.306377


In [15]:
targetGroups_df = caprovider_df.loc[(caprovider_df["Provider_Type_Code_Desc"] == "PHYSICIANS GROUP (FFS)") | (caprovider_df["Provider_Type_Code_Desc"] == "PHYSICIANS (FFS)")]
targetGroups_df = targetGroups_df.loc[(caprovider_df["Provider_Address_County_Code_De"] != "UNKNOWN") & ]
targetGroups_df.head(10)

,Provider_Number,Provider_Type_Code_Desc,Provider_Specialty_Code_Desc,Provider_Address_County_Code_De,Provider_Address_Latitude,Provider_Address_Longitude
3,1003121930,PHYSICIANS (FFS),Internal Medicine,Humboldt,40.783220,-124.142048
8,1003122813,PHYSICIANS (FFS),Emergency Medicine (Urgent Care),Stanislaus,37.706404,-121.050079
10,1003123670,PHYSICIANS (FFS),Family Practice,Los Angeles,34.098161,-118.292966
12,1003126327,PHYSICIANS (FFS),General Practice,San Francisco,37.763821,-122.456707
18,1003127234,PHYSICIANS (FFS),Anesthesiology,Orange,33.847152,-117.933932
19,1003127242,PHYSICIANS (FFS),Internal Medicine,Monterey,36.576426,-121.913993
20,1003127846,PHYSICIANS (FFS),Internal Medicine,Santa Clara,37.433373,-122.177575
22,1003127986,PHYSICIANS (FFS),Internal Medicine,Santa Clara,37.239456,-121.802937
23,1003128323,PHYSICIANS (FFS),Anesthesiology,Los Angeles,34.076824,-118.380276
24,1003128364,PHYSICIANS (FFS),Emergency Medicine (Urgent Care),Alameda,37.697452,-122.087053


In [13]:
targetGroups_df = targetGroups_df.rename(columns={"Provider_Number": "Provider",
                                                "Provider_Type_Code_Desc": "Provider_Type",
                                                "Provider_Specialty_Code_Desc": "Specialty",
                                                "Provider_Address_County_Code_De": "County_Code"
                                                })
targetGroups_df.head()

,Provider,Provider_Type,Specialty,County_Code,Provider_Address_Latitude,Provider_Address_Longitude
3,1003121930,PHYSICIANS (FFS),Internal Medicine,Humboldt,40.783220,-124.142048
8,1003122813,PHYSICIANS (FFS),Emergency Medicine (Urgent Care),Stanislaus,37.706404,-121.050079
10,1003123670,PHYSICIANS (FFS),Family Practice,Los Angeles,34.098161,-118.292966
12,1003126327,PHYSICIANS (FFS),General Practice,San Francisco,37.763821,-122.456707
18,1003127234,PHYSICIANS (FFS),Anesthesiology,Orange,33.847152,-117.933932


In [6]:
targetCounts = targetGroups_df.value_counts('County_Code', sort=False)
targetCounts

County_Code
Alameda             6164
Alpine                 1
Amador                98
Butte                716
Calaveras             63
Colusa                37
Contra Costa        3801
Del Norte             73
El Dorado            410
Fresno              3032
Glenn                 37
Humboldt             611
Imperial             393
Inyo                  77
Kern                2619
Kings                289
Lake                 168
Lassen                76
Los Angeles        39355
Madera               528
Marin                854
Mariposa              20
Mendocino            292
Merced               564
Modoc                 10
Mono                  53
Monterey            1252
Napa                 425
Nevada               276
Orange             10510
Placer              1302
Plumas                32
Riverside           5143
Sacramento          7572
San Benito            83
San Bernardino      6950
San Diego          13348
San Francisco       8324
San Joaquin         2289
San Luis Obis

In [7]:
m = targetCounts.median()
m

564.0

In [8]:
targetsGrouped_df = targetGroups_df.groupby('County_Code').agg({'Provider_Address_Latitude':'mean', 'Provider_Address_Longitude':'mean'})
targetsGrouped_df.head(60)

,Provider_Address_Latitude,Provider_Address_Longitude
County_Code,,
Alameda,37.742428,-122.155974
Alpine,38.769184,-119.815993
Amador,38.355900,-120.767838
Butte,39.680714,-121.731512
Calaveras,38.171025,-120.626371
Colusa,39.205536,-122.004586
Contra Costa,37.938350,-122.063218
Del Norte,41.774686,-124.193364
El Dorado,38.802324,-120.469627


In [9]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

In [10]:
locations = targetsGrouped_df[["Provider_Address_Latitude", "Provider_Address_Longitude"]].astype(float)

heat_layer = gmaps.heatmap_layer(locations, weights=targetCounts, 
                                 dissipating=False, max_intensity=3000,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))